In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import datetime
import calendar
import re

In [2]:
imdb = pd.read_csv('movie_bd_v5.csv')
imdb.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1724,tt0890870,10000000,139352633,Saw IV,Tobin Bell|Scott Patterson|Costas Mandylor|Bet...,Darren Lynn Bousman,It's A Trap,Jigsaw and his apprentice Amanda are dead. Now...,93,Horror|Thriller|Crime,Lions Gate Films|Twisted Pictures,10/25/2007,5.8,2007
206,tt2870612,5000000,40100000,"As Above, So Below",Perdita Weeks|Ben Feldman|Edwin Hodge|FranÃ§oi...,John Erick Dowdle,The only way out is down.,When a team of explorers ventures into the cat...,93,Horror|Thriller,Legendary Pictures,8/14/2014,6.1,2014
1614,tt0363988,40000000,92913171,Secret Window,Johnny Depp|John Turturro|Timothy Hutton|Charl...,David Koepp,The most important part of a story is the ending.,"Mort Rainey, a writer just emerging from a pai...",96,Thriller|Mystery,Columbia Pictures Corporation|Pariah Entertain...,3/12/2004,6.3,2004
685,tt1032755,18000000,25739015,RockNRolla,Mark Strong|Tom Wilkinson|Toby Kebbell|Gerard ...,Guy Ritchie,"A story of sex, thugs and rock 'n roll.",When a Russian mobster sets up a real estate s...,114,Action|Crime|Thriller,StudioCanal|Dark Castle Entertainment|Toff Guy...,9/4/2008,6.8,2008
645,tt0770752,70000000,111231041,Fool's Gold,Matthew McConaughey|Kate Hudson|Donald Sutherl...,Andy Tennant,This February True Love Takes a Dive.,Charming but luckless treasure hunter Ben Finn...,112,Comedy,De Line Pictures|Warner Bros.,2/7/2008,5.3,2008


In [3]:
imdb.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

def extend_imdb(df, column, extension_column):
    extension = pd.DataFrame(df[column].str.split("|").tolist(), index=df.imdb_id).stack()
    extension = extension.reset_index([0, "imdb_id"])
    extension.columns = ['imdb_id', extension_column]
    return df.merge(extension, on="imdb_id", how="left")

imdb['profit'] = imdb['revenue'] - imdb['budget']
imdb['month'] = imdb.release_date.apply(lambda rd: datetime.datetime.strptime(str(rd), "%m/%d/%Y").month)
imdb['title_length'] = imdb.original_title.apply(lambda t: len(t))
imdb['overview_words'] = imdb.overview.apply(lambda o: len(re.findall(r"[\w']+", o.lower())))

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
result = imdb[imdb.budget == imdb.budget.max()].head(1)
answers['1'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
result = imdb[imdb.runtime == imdb.runtime.max()].head(1)
answers['2'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
result = imdb[imdb.runtime == imdb.runtime.min()].head(1)
answers['3'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = "%d" % round(imdb['runtime'].mean())

# 5. Каково медианное значение длительности фильмов? 

In [9]:
answers['5'] = "%d" % round(imdb['runtime'].median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
result = imdb[imdb.profit == imdb.profit.max()].head(1)
answers['6'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 7. Какой фильм самый убыточный? 

In [11]:
result = imdb[imdb.profit == imdb.profit.min()].head(1)
answers['7'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
result = len(imdb[imdb.profit > 0].index)
answers['8'] = "%d" % result

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
imdb_for_2008 = imdb[imdb.release_year == 2008]
result = imdb_for_2008[imdb_for_2008.profit == imdb_for_2008.profit.max()].head(1)
answers['9'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
imdb_from2012_to_2014 = imdb[(imdb.release_year >= 2012) & (imdb.release_year <= 2014)]
result = imdb_from2012_to_2014[imdb_from2012_to_2014.profit == imdb_from2012_to_2014.profit.min()].head(1)
answers['10'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 11. Какого жанра фильмов больше всего?

In [15]:
imdb_with_genres = extend_imdb(imdb, "genres", "genre")
answers['11'] = imdb_with_genres['genre'].value_counts().index[0]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [16]:
imdb_with_genres = extend_imdb(imdb, "genres", "genre")
result = imdb_with_genres[imdb_with_genres.profit > 0]['genre'].value_counts()
answers['12'] = result.index[0]

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [17]:
imdb_with_directors = extend_imdb(imdb, "director", "director_single")
result = imdb_with_directors.groupby('director_single')['revenue'].sum().sort_values(ascending=False).head(1)
answers['13'] = result.index[0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [18]:
imdb_actions_with_director = extend_imdb(imdb[imdb.genres.str.contains('Action')], "director", "director_single")
result = imdb_actions_with_director.groupby('director_single')['imdb_id'].count().sort_values(ascending=False).head(1)
answers['14'] = result.index[0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [19]:
imdb_for_2012_with_actors = extend_imdb(imdb[imdb.release_year == 2012], "cast", "actor")
result = imdb_for_2012_with_actors.groupby('actor')['revenue'].sum().sort_values(ascending=False).head(1)
answers['15'] = result.index[0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [20]:
imdb_high_budget_with_actors = extend_imdb(imdb[imdb.budget > imdb.budget.mean()], "cast", "actor")
result = imdb_high_budget_with_actors['actor'].value_counts().head(1)
answers['16'] = result.index[0]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [21]:
imdb_with_cage_and_genres = extend_imdb(imdb[imdb.cast.str.contains('Nicolas Cage')], "genres", "genre")
result = imdb_with_cage_and_genres['genre'].value_counts().head(1)
answers['17'] = result.index[0]

# 18. Самый убыточный фильм от Paramount Pictures

In [22]:
imdb_with_paramount = imdb[imdb.production_companies.str.contains("Paramount Pictures")]
result = imdb_with_paramount[imdb_with_paramount.profit == imdb_with_paramount.profit.min()].head(1)
answers['18'] = "%s (%s)" % (result['original_title'].item(), result['imdb_id'].item())

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [23]:
result = imdb.groupby('release_year')['revenue'].sum().sort_values(ascending=False).head(1)
answers['19'] = str(result.index[0])

# 20. Какой самый прибыльный год для студии Warner Bros?

In [24]:
imdb_with_wb = imdb[imdb.production_companies.str.contains("Warner Bros.")]
result = imdb_with_wb.groupby('release_year')['profit'].sum().sort_values(ascending=False).head(1)
answers['20'] = str(result.index[0])

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [25]:
result = imdb['month'].value_counts().head(1)
answers['21'] = calendar.month_name[result.index[0]]
# тут, конечно, было бы хорошо вывести на русском, чтоб полностью совпадало, но не успеваю :(

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [26]:
answers['22'] = str(imdb[(imdb.month >= 6) & (imdb.month <= 8)]['imdb_id'].count())

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [27]:
imdb_on_winter = imdb[(imdb.month == 1) | (imdb.month == 2) | (imdb.month == 12)]
imdb_on_winter_with_director = extend_imdb(imdb_on_winter, "director", "director_single")
result = imdb_on_winter_with_director['director_single'].value_counts().head(1)
answers['23'] = result.index[0]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [28]:
imdb_with_company = extend_imdb(imdb, 'production_companies', 'company')
result = imdb_with_company.groupby('company')['title_length'].mean().sort_values(ascending=False).head(1)
answers['24'] = result.index[0]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [29]:
imdb_with_company = extend_imdb(imdb, 'production_companies', 'company')
result = imdb_with_company.groupby('company')['overview_words'].mean().sort_values(ascending=False).head(1)
answers['25'] = result.index[0]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [30]:
variants = [
    "Inside Out, The Dark Knight, 12 Years a Slave",
    "BloodRayne, The Adventures of Rocky & Bullwinkle",
    "Batman Begins, The Lord of the Rings: The Return of the King, Upside Down",
    "300, Lucky Number Slevin, Kill Bill: Vol. 1",
    "Upside Down, Inside Out, Iron Man"
]
result = ""
top_movies = list(imdb.sort_values("vote_average", ascending=False).head(round(len(imdb.index) / 100))['original_title'])
for v in variants:
    movies = v.split(sep=", ")
    if len(movies) == len([s for s in movies if s in top_movies]):
        result = v
        break

answers['26'] = result

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [31]:
variants = [
    "Johnny Depp & Helena Bonham Carter",
    "Ben Stiller & Owen Wilson",
    "Vin Diesel & Paul Walker",
    "Adam Sandler & Kevin James",
    "Daniel Radcliffe & Rupert Grint"
]
# Это не самый эфективный способ, так как для каждой пары происходит отдельный поиск по всему датафрейму
# Эффективней подсчитывать сразу в один проход, но это фактически уже просто проход по массиву с подсчетом,
#  поэтому сделал таким образом, просто ради интереса
c = Counter()
for v in variants:
    pair = v.split(sep=" & ")
    c[v] = len(imdb[(imdb.cast.str.contains(pair[0])) & (imdb.cast.str.contains(pair[1]))].index)
answers['27'] = c.most_common(1)[0][0]

# Submission

In [32]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [33]:
len(answers)

27

В шаблоне предлагалось ответы прописывать явно и отдельно решения того, как получили ответ.
Мне это показалось странным, поэтому я словарь ответов заполнял сразу при помощи кода (все приведя к строкам для единообразия)